In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
train_data.head()

In [ ]:
train_x = train_data.drop('label',axis = 1)
train_y = train_data['label']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
sample_arr = train_x.sample(n = 1,random_state = np.random.RandomState()).values
random_sample_image = sample_arr.reshape((28,28,1))
plt.imshow(random_sample_image)

In [ ]:

train_y.value_counts().plot.bar()

In [ ]:
import torch

In [ ]:
class  create_dataset():
    def __init__(self,data,label = None,is_test = False):
        self.data = data/255.0
        self.label= label                            
        self.is_test = is_test
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,idx):
        sample = self.data[idx,:]
        if self.is_test is False:
            label = self.label[idx]
            return (
            torch.tensor(sample, dtype = torch.float),
            torch.tensor(label, dtype = torch.long)
            )
        else:
            torch.tensor(self.data, dtype = torch.float)
        

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,Y_train,y_val = train_test_split(train_x.values,train_y.values,test_size = 0.2,random_state = 1)

In [ ]:
train_ds = create_dataset(X_train,Y_train)
val_ds = create_dataset(X_val,y_val)


In [ ]:
class model_config:
    lr = 1e-3
    epochs = 5
    train_batch = 64
    test_batch = 64
    val_batch = 64

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size = model_config.train_batch)

val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size = model_config.val_batch)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    
    break

In [ ]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

In [ ]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28,512),
                                       nn.ReLU(),
                                       nn.Linear(512,512),
                                       nn.ReLU(),
                                       nn.Dropout(0.2),
                                       nn.Linear(512,256),
                                       nn.ReLU(),
                                       nn.Dropout(0.1),
                                       nn.Linear(256,10)
                                       )

    def forward(self, X):
        
        logits = self.linear_relu_stack(X)
        return logits

model = NeuralNetwork().to(device)
print(model)
    

In [ ]:
#to Train the model we require loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = model_config.lr)

In [ ]:

def train_model(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_size,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_size%100 == 0:
            loss,current = loss.item(),batch_size*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
epoch = 5 
for t in range(epoch):
    print(f"epoch {t+1}\n--------------------------")
    train_model(train_dataloader,model,loss_fn,optimizer)
    
print("Done!")

In [ ]:
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_data.shape

In [ ]:
test_data.isnull().values.sum()

In [ ]:
test = test_data.values 
test_ds = torch.tensor(test, dtype = torch.float)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size = model_config.val_batch)

In [ ]:
test.shape

In [ ]:
from tqdm import tqdm
#preparing results
results = None
with torch.no_grad():
    for X in tqdm(test_dataloader):
        X = X.to(device)
        pred = model(X)
        if results == None:
                results = pred
        else:
                results = torch.cat((results, pred), 0)
        print(results.shape)

In [ ]:
print(results.shape)

In [ ]:
np_pred = results.cpu().detach().numpy()
y_pred = np.argmax(np_pred, axis=1)

In [ ]:
submission = pd.DataFrame(
        {'ImageId': [i for i in range(1, len(np_pred)+1)], 'Label': y_pred})
    # saves the submission file
submission.to_csv('submission.csv', index=False)
   

In [ ]:
submission.head()